# Tutorial for Searching a STAC Catalog

In this tutorial you will learn how to connect to a STAC API and search various datasets available for query. You will also search for specific items using a range of query parameters.  

We will use the [Earth Search API](https://www.element84.com/earth-search/) developed by Element84 for satellite datasets available on AWS S3 Storage (note that only those datasets that have a STAC catalog are served through this API). 
You can access the STAC catalog of the API [here](https://earth-search.aws.element84.com/v1). 

We will use the PySTAC pakcage for this tutorial. Check the documentation [here](https://pystac.readthedocs.io/en/stable/), and [these](https://pystac-client.readthedocs.io/en/latest/usage.html#itemsearch) specific guides about using ItemSearch

***Attribution***: Parts of this notebook are inspired by the great tutorial on **Access satellite imagery using Python** ([link](https://carpentries-incubator.github.io/geospatial-python/instructor/05-access-data.html#search-a-stac-catalog))

In [ ]:
from pystac_client import Client

In [ ]:
api_url = "https://earth-search.aws.element84.com/v1"

In [ ]:
client = Client.open(api_url)

## Find Collections

First, we would like to see what collections are available from this API. 

In [ ]:
collections = client.get_collections()

In [ ]:
for collection in collections:
    print(collection)

In order to get more information about a specific collection, you can use `get_collection` function:

In [ ]:
s1_collection = client.get_collection("sentinel-1-grd")
s1_collection

## Search Items

Let's use Leafmap to select a point where we are interested to find a satellite imagery

In [ ]:
import leafmap

In [ ]:
m = leafmap.Map(center=[42.250809, -71.822833], zoom=16, height="800px")
m

Pan and zoom the map to find an area of interest, then use the tools on the top left of the map to select a point on the map. 


In [ ]:
if m.user_rois is not None:
    point = m.user_rois['features'][0]['geometry']
else:
    point = dict(type="Point", coordinates=(42.250809, -71.822833))

We are interested to search for Sentinel-2 imagery that intersects with the point we selected in the previous step. So we will use the `search` function and insert `sentinel-2-l2a` as our collection of interest. We also use the `intersects` property to filter the scenes that intersect with our point of interest. 


In [ ]:
search_results = client.search(
    collections=["sentinel-2-l2a"],
    intersects=point,
    max_items=12,
)

In [ ]:
print(search_results.matched())

The number that is returned in the previous step is more than 12 that we had identified in our search criteria. But this doesn't mean all the metadata about these scenes haven been retrieved. This just shows how many scenes have `matched` our search criteria. In the next cell, we will call `item_collection()` to retrieve the metadata, and check how many of them are retrieved. 

In [ ]:
items = search_results.item_collection()

In [ ]:
len(items)

Now, let's investigate an item

In [ ]:
items[0]

In [ ]:
print(items[0].datetime)

In [ ]:
print(items[0].geometry)

We can now use the item's geometry and confirm that the returned scene intersects with our point of interest. 

In [ ]:
m.add_geojson(items[0].geometry)
m

## Query Metadata

Items in STAC catalog have much more metadata (in addition to location) that you can query and only return results that match your query parameters. 
Let's use the `datetime` property and only search for scenes in 2023:

In [ ]:
search = client.search(
    collections=["sentinel-2-l2a"],
    intersects=point,
    datetime="2023-01-01/2023-09-22"
)

In [ ]:
print(search.matched())

Another property which is key for multispectral imagery is cloud cover; ideally we would be interested to find scenes with low cloud cover. Cloud cover is recorded in the metadata named `eo:cloud_cover`, and it ranges from 0 to 1. In the following, we are going to find scenes that only have a cloud cover of less than 5% in 2023.

In [ ]:
search = client.search(
    collections=["sentinel-2-l2a"],
    intersects=point,
    datetime="2023-01-01/2023-09-22",
    query=["eo:cloud_cover<5"],
    max_items=10
)

In [ ]:
print(search.matched())

Another useful property of the STAC API is that you can sort the results using a metadata property. For example, let's sort our results based on the cloud cover value:

In [ ]:
search = client.search(
    collections=["sentinel-2-l2a"],
    intersects=point,
    datetime="2023-01-01/2023-09-22",
    query=["eo:cloud_cover<5"],
    sortby=["+properties.eo:cloud_cover"],
    max_items=10
)

In [ ]:
items = search.item_collection()

In [ ]:
len(items)

In [ ]:
for item in items:
    print(item.properties["eo:cloud_cover"])

You can also save the results of the search into a JSON file if you need it later on. 

In [ ]:
items.save_object("search.json")

## Access Assets

Next, we will use the assets of the returned items and retrieve the actual scene. 

In [ ]:
#Let's look at the second item:
selected_item = items[1]

In [ ]:
# Here are the assets available for this item
assets = selected_item.assets
print(assets.keys())

In [ ]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

Each asset has a link which can be accessed from the `href` property. This can be a HTTP URL or a link to S3 for this STAC catalog. For example, let's look at the link for the thumbnail of the scene.  

In [ ]:
print(assets["thumbnail"].href)

Since this is a HTTP link, we can use Python requests package to load the image. 

In [ ]:
import requests
img_data = requests.get(assets["thumbnail"].href).content

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import io
plt.figure(figsize=(10, 10))
plt.imshow(Image.open(io.BytesIO(img_data)))

Now, let's load one of the COG assets into memory. This is a large scene, and we don't necessary want to load all the data at once. There are various packages to do this. Here we will use `rioxarray`, you can also use [`stacstack`](https://stackstac.readthedocs.io/en/latest/) and [`odc-stac`](https://odc-stac.readthedocs.io/).

In [ ]:
import rioxarray

In [ ]:
nir_href = assets["nir"].href
nir = rioxarray.open_rasterio(nir_href)
nir

**Note**: At this stage the `nir` DataArray that we defined is empty, and only the metadata of the scene is loaded from the target href. This is consistent with `open_rasterio` function behavior, and you will learn more about it later in the class. 
When you run any function on `nir` or call any of the built-in function (e.g. `mean()`) the data will be loaded to memory. 

First, let's plot part of the `nir` array. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(nir[0,1500:2000,6200:7000], cmap='gray')

In this case, only the portion of the array that we wanted to visualize was loaded. 

We can actually time the operation of plotting the whole scene vs the small portion we selected, and see the efficiency of using this approach. 

In [ ]:
%%timeit -n 5 -r 1
plt.imshow(nir[0,1500:2000,6200:7000], cmap='gray')

In [ ]:
%%timeit -n 5 -r 1
plt.imshow(nir[0, :, :], cmap='gray')

Lastly, you can save this array into a GeoTIFF file as following:

In [ ]:
nir[0,1500:2000,6200:7000].rio.to_raster("nir_subset.tif")